<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i> 유방암 데이터를 통한 앙상블_부스팅 실습
  </h2>
</div>


In [1]:
from sklearn.metrics import confusion_matrix # 오차행렬
from sklearn.metrics import accuracy_score # 정확도
from sklearn.metrics import precision_score # 정밀도
from sklearn.metrics import recall_score # 재현율
from sklearn.metrics import roc_auc_score # auc score
from sklearn.metrics import f1_score 

def get_clf_eval(y_test,y_pred):
    matrix = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)
    auc = roc_auc_score(y_test,y_pred)
    
    print('{}'.format('RESULT'))
    print('Confusion Matrix : ')
    print(matrix)
    print('-'*30)
    
    print('Accuaracy Score : ')
    print(accuracy)
    print('-'*30)
    
    print('Precision Score : ')
    print(precision)
    print('-'*30)
    
    print('Recall Score : ')
    print(recall)
    print('-'*30)
    
    print('F1 Score : ')
    print(f1)
    print('-'*30)
    
    print('AUC Score : ')
    print(auc)
    print('-'*30)

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# mpl.rcParams['font.family'] = 'AppleGothic'

In [3]:
cancer = load_breast_cancer() # 데이터를 불러옴 
# 데이터를 프레임으로 만듬 
cancer_df = pd.DataFrame(data=cancer.data,
                         columns=cancer.feature_names)
# Y label == target 
cancer_df['target'] = cancer.target
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, # cancer_df.iloc[:,:-1] 혹은 cancer_df.drop('target',axis=1)
                                                    y, # cancer_df.target
                                                    test_size=0.3,
                                                   random_state=42)

---

## Boosting

- 여러 부스팅 알고리즘


```python 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
```

<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  GradientBoostingClassifier
  </h2>
</div>


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
gbm_clf = GradientBoostingClassifier() # 정의 
gbm_clf.fit(X_train,y_train) # 학습 

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [6]:
gbm_clf.score(X_test,y_test) #평가 

0.9590643274853801

<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  XGBClassifier
  </h2>
</div>


In [7]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier() # 정의 
xgb_clf.fit(X_train,y_train)# 학습 
xgb_clf.score(X_test,y_test) # 평가 

0.9824561403508771

In [8]:
xgb_pred = xgb_clf.predict(X_test)

In [9]:
# 조기 중단 기능 (early stopping) 활용
eval_data = [(X_test,y_test)]
xgb_clf = XGBClassifier(n_estimators=500)
# xgb_clf.fit(X_train,y_train)# 학습 
xgb_clf.fit(X_train,y_train,
            early_stopping_rounds= 100,
            eval_set = eval_data,
            eval_metric = 'logloss')

[0]	validation_0-logloss:0.47549
Will train until validation_0-logloss hasn't improved in 100 rounds.
[1]	validation_0-logloss:0.35952
[2]	validation_0-logloss:0.28211
[3]	validation_0-logloss:0.23664
[4]	validation_0-logloss:0.19574
[5]	validation_0-logloss:0.16801
[6]	validation_0-logloss:0.14878
[7]	validation_0-logloss:0.13211
[8]	validation_0-logloss:0.12492
[9]	validation_0-logloss:0.10982
[10]	validation_0-logloss:0.10060
[11]	validation_0-logloss:0.09939
[12]	validation_0-logloss:0.09412
[13]	validation_0-logloss:0.08917
[14]	validation_0-logloss:0.08651
[15]	validation_0-logloss:0.08647
[16]	validation_0-logloss:0.08490
[17]	validation_0-logloss:0.08154
[18]	validation_0-logloss:0.07955
[19]	validation_0-logloss:0.07847
[20]	validation_0-logloss:0.07677
[21]	validation_0-logloss:0.07694
[22]	validation_0-logloss:0.07704
[23]	validation_0-logloss:0.07597
[24]	validation_0-logloss:0.07579
[25]	validation_0-logloss:0.07556
[26]	validation_0-logloss:0.07567
[27]	validation_0-loglo

[236]	validation_0-logloss:0.06993
[237]	validation_0-logloss:0.06993
[238]	validation_0-logloss:0.06993
[239]	validation_0-logloss:0.06993
[240]	validation_0-logloss:0.06993
[241]	validation_0-logloss:0.06993
[242]	validation_0-logloss:0.06993
[243]	validation_0-logloss:0.06993
[244]	validation_0-logloss:0.06993
Stopping. Best iteration:
[144]	validation_0-logloss:0.06993



XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  AdaBoostClassifier
  </h2>
</div>


In [10]:
from sklearn.ensemble import AdaBoostClassifier
ab_model = AdaBoostClassifier() # 정의
ab_model.fit(X_train, y_train) # 학습 
y_pred = ab_model.predict(X_test) #예측
get_clf_eval(y_test,y_pred ) # 평가 (사용자정의함수)

RESULT
Confusion Matrix : 
[[ 61   2]
 [  2 106]]
------------------------------
Accuaracy Score : 
0.9766081871345029
------------------------------
Precision Score : 
0.9814814814814815
------------------------------
Recall Score : 
0.9814814814814815
------------------------------
F1 Score : 
0.9814814814814815
------------------------------
AUC Score : 
0.9748677248677248
------------------------------


<div class="alert alert-success" data-title="">
  <h2><i class="fa fa-tasks" aria-hidden="true"></i>  LGBMClassifier
  </h2>
</div>


In [11]:
from lightgbm import LGBMClassifier
lightgbm = LGBMClassifier() # 모델정의 
lightgbm.fit(X_train,y_train)# 정의된 모델 학습 
y_pred = lightgbm.predict(X_test) # 예측 
get_clf_eval(y_test,y_pred)# 평가 

RESULT
Confusion Matrix : 
[[ 59   4]
 [  2 106]]
------------------------------
Accuaracy Score : 
0.9649122807017544
------------------------------
Precision Score : 
0.9636363636363636
------------------------------
Recall Score : 
0.9814814814814815
------------------------------
F1 Score : 
0.9724770642201834
------------------------------
AUC Score : 
0.958994708994709
------------------------------
